In [1]:
import requests, json
from tqdm import tqdm
from bs4 import BeautifulSoup
import urllib.parse as parse
import pandas as pd

In [2]:
url = 'https://search.naver.com/search.naver?'
qs = 'where=news&sm=tab_pge&query=러시아%20한국%20경제&sort=1&photo=0&field=0&pd=3&ds=2022.02.24&de=2022.04.15&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from20220224to20220415,a:all&start=2581'
params = dict(parse.parse_qsl(qs))
params['cluster_rank'] = ''
params

'where=news&sm=tab_pge&query=러시아%20한국%20경제&sort=1&photo=0&field=0&pd=3&ds=2022.02.24&de=2022.04.15&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from20220224to20220415,a:all&start=2581'
'where=news&sm=tab_pge&query=러시아%20한국%20경제&sort=1&photo=0&field=0&pd=3&ds=2022.02.24&de=2022.04.15&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from20220224to20220415,a:all&start=1'

{'cluster_rank': '',
 'de': '2022.04.15',
 'ds': '2022.02.24',
 'field': '0',
 'mynews': '0',
 'nso': 'so:dd,p:from20220224to20220415,a:all',
 'office_section_code': '0',
 'office_type': '0',
 'pd': '3',
 'photo': '0',
 'query': '러시아 한국 경제',
 'sm': 'tab_pge',
 'sort': '1',
 'start': '2581',
 'where': 'news'}

In [9]:
def articlesAdrsLstGen(keywords):
  adrs_lst = []

  for page in tqdm(range(1, 2582, 10)):
    try:
        headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
        }
        url = 'https://search.naver.com/search.naver?'
        qs = 'where=news&sm=tab_pge&query=러시아%20한국%20경제&sort=1&photo=0&field=0&pd=3&ds=2022.02.24&de=2022.04.15&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from20220224to20220415,a:all&start=2581'
        params = dict(parse.parse_qsl(qs))
        params['cluster_rank'] = ''
        params['query'] = keywords
        params['start'] = page
        resp = requests.get(url, headers=headers, params=params)
        soup = BeautifulSoup(resp.content, 'lxml')
        adrs = soup.select('div.news_info > div.info_group > a')
        for item in adrs:
          if 'https://news.naver.com/main/read.naver?' in item['href'] and item['href'] not in adrs_lst:
            adrs_lst.append(item['href'])
    except Exception as e:
      print()
      print(e)
      print(item['href'])
      continue
  return adrs_lst

# 'div.news_wrap.api_ani_send > div > div.news_info > div.info_group > a
# '#sp_nws45 > div.news_wrap.api_ani_send > div > div.news_info > div.info_group > a:nth-child(3)'
# '#sp_nws50 > div > div > div.news_info > div.info_group > a:nth-child(3)'

def articlesTableGen(adrs_lst):
  articles = {
    '등록일시': [],
    '제목': [],
    '내용': [],
    '주소': [],
  }

  for adrs in tqdm(adrs_lst):
      try:
        headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
        }
        url = adrs
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.content, 'lxml')
        get_title = soup.select('#articleTitle')
        get_time = soup.select('#main_content > div.article_header > div.article_info > div > span.t11')
        get_content = soup.select('#articleBodyContents')
        articles['등록일시'].append(get_time[0].text)
        articles['제목'].append(get_title[0].text)
        articles['내용'].append(get_content[0].text)
        articles['주소'].append(adrs)
      except Exception as e:
        print()
        print(e)
        print('오류주소', adrs)
        continue
  table = pd.DataFrame(articles)
  return table


In [10]:
lst = articlesAdrsLstGen('러시아 한국 경제')

100%|██████████| 259/259 [05:46<00:00,  1.34s/it]


In [11]:
len(lst)

1456

In [12]:
len(set(lst))

1456

In [ ]:
lst

In [14]:
df = articlesTableGen(lst)

100%|██████████| 1456/1456 [14:55<00:00,  1.63it/s]


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   등록일시    1456 non-null   object
 1   제목      1456 non-null   object
 2   내용      1456 non-null   object
 3   주소      1456 non-null   object
dtypes: object(4)
memory usage: 45.6+ KB


In [16]:
df

,등록일시,제목,내용,주소
0,2022.04.15. 오후 7:51,"""오늘 점심 칼국수 어때?"" 이런말 못하겠네…서울서 8000원 첫 돌파",\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
1,2022.04.15. 오후 7:28,"""우크라 전쟁 장기화""…농업株 일제히 강세",\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
2,2022.04.15. 오후 5:50,식량자급률 내년부터 꺾인다,\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
3,2022.04.15. 오후 5:41,"윤석열 당선인 ""심상치 않다…경제 복합위기 징후 뚜렷""",\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
4,2022.04.15. 오후 5:38,"尹 ""더 강한 한미동맹 바탕…유럽·아시아로 외교 확장""",\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
...,...,...,...,...
1451,2022.02.24. 오전 8:01,[전국 주요 신문 톱뉴스](24일 조간),\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
1452,2022.02.24. 오전 7:41,[조간브리핑] 고신용자도 대출 금리 4% 넘어서…본격적 ‘고금리 부담’ 시작,\n\n\n\n동영상 뉴스\n\n\n\n\n// flash 오류를 우회하기 위한 함...,https://news.naver.com/main/read.naver?mode=LS...
1453,2022.02.24. 오전 6:23,사라지지 않은 러시아 전면 침공 가능성···미 기준금리 전망도 움직이나 [글로벌마켓...,\n\n\n\n동영상 뉴스\n\n\n\n\n// flash 오류를 우회하기 위한 함...,https://news.naver.com/main/read.naver?mode=LS...
1454,2022.02.24. 오전 3:02,"국제유가 장중 99달러… 한국, 3%성장 ‘빨간불’",\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...


In [17]:
cond = df.duplicated()
df[cond].count()

등록일시    0
제목      0
내용      0
주소      0
dtype: int64

In [20]:
df.to_csv('러시아 한국 경제_v2_2022041520220224.csv', index=False)